In [3]:
from make_prediction import DataSplits

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
data = pd.read_pickle("predictions.pkl")

,RichDLLe,RichDLLk,RichDLLmu,RichDLLp,RichDLLbt,Brunel_P,Brunel_ETA,nTracks_Brunel,probe_sWeight,predicted_RichDLLe,predicted_RichDLLk,predicted_RichDLLmu,predicted_RichDLLp,predicted_RichDLLbt
0,-10.894900,-108.897000,7.334450,-87.995400,-87.995400,13990.50,2.96764,91,0.922170,-2.834047,-56.128766,5.935415,-47.138826,-47.056507
1,-51.910400,-48.369800,-10.189800,-48.369800,-48.369800,7161.77,2.25095,214,-0.348688,-15.931120,-31.700560,13.260075,-31.733942,-31.709250
2,-0.375961,-5.019910,-0.146470,-22.590900,-21.257700,71984.60,3.33840,168,0.832851,-0.882784,-8.008081,-0.176931,-54.605153,-62.603282
3,-1.885830,16.302200,0.094440,16.302200,16.302200,8905.33,3.71419,373,0.923306,-8.273911,-2.063076,0.608423,-0.688363,-0.369104
4,-2.042050,-58.811800,0.785152,-46.346100,-45.548700,23658.90,3.72172,177,1.247080,-4.155358,-24.883122,-1.279007,-31.083448,-23.259552
5,0.007245,-6.417200,0.010756,-23.363300,-24.948300,88301.70,4.24548,194,0.001446,-0.506469,-3.659191,-0.177322,-29.662179,-32.819709
6,-7.505040,-23.372100,6.587820,-23.372100,-23.372100,7456.89,2.82590,279,1.039760,-14.379708,-36.092556,5.971710,-35.655480,-35.719047
7,3.145450,-4.316830,0.792948,27.410100,17.368100,23619.10,3.60085,311,-0.264425,-4.243313,-13.105894,-2.258069,-16.934411,-13.220151
8,-15.017200,1.414800,-1.106590,1.414800,1.414800,8340.35,3.71479,276,-0.362472,-28.965840,-25.881947,-2.637787,-26.406034,-26.350619
9,-1.641090,-40.304000,-0.910127,-46.460000,-40.797700,40874.90,3.36335,49,1.251810,0.000000,-49.615651,0.021892,-86.962470,-63.930295
